# Testing graph isomorphism agents

## Setup

In [1]:
FORCE_CPU = True

SEED = 349287

LOAD_CHECKPOINT = False
CHECKPOINT_RUN_ID = "ppo_gi_fixed_shared_min_2_2_3"
CHECKPOINT_VERSION = "v0"
NORMALIZE_MESSAGE_HISTORY_DEFAULT = True

BATCH_SIZE = 128

In [2]:
import torch
from torch import Tensor
from torch import nn

from tensordict import TensorDictBase
from tensordict.nn import TensorDictSequential

import numpy as np

from tqdm import tqdm

import pandas as pd

import plotly.graph_objects as go
import plotly.express as px

from rich.console import Console
from rich.table import Table

from pvg import (
    Parameters,
    ScenarioType,
    TrainerType,
    AgentsParameters,
    GraphIsomorphismAgentParameters,
)
from pvg.experiment_settings import ExperimentSettings
from pvg.graph_isomorphism import (
    GraphIsomorphismAgentHooks,
)
from pvg.scenario_instance import build_scenario_instance
from pvg.scenario_base import DataLoader

In [3]:
torch.manual_seed(SEED)
torch_generator = torch.Generator().manual_seed(SEED)

In [4]:
if not FORCE_CPU and torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

cpu


## Create scenario and agents

In [5]:
params = Parameters(
    scenario=ScenarioType.GRAPH_ISOMORPHISM,
    trainer=TrainerType.VANILLA_PPO,
    dataset="eru10000",
    seed=SEED,
    agents=AgentsParameters(
        verifier=GraphIsomorphismAgentParameters(
            load_checkpoint_and_parameters=LOAD_CHECKPOINT,
            checkpoint_run_id=CHECKPOINT_RUN_ID,
            checkpoint_version=CHECKPOINT_VERSION,
            normalize_message_history=NORMALIZE_MESSAGE_HISTORY_DEFAULT,
        ),
        prover=GraphIsomorphismAgentParameters(
            load_checkpoint_and_parameters=LOAD_CHECKPOINT,
            checkpoint_run_id=CHECKPOINT_RUN_ID,
            checkpoint_version=CHECKPOINT_VERSION,
            normalize_message_history=NORMALIZE_MESSAGE_HISTORY_DEFAULT,
        ),
    ),
)

In [6]:
settings = ExperimentSettings(
    device=device,
)

In [7]:
scenario_instance = build_scenario_instance(
    params=params,
    settings=settings,
)

In [8]:
class TensorDictSequentialHooks(TensorDictSequential):
    def forward(
        self,
        tensordict: TensorDictBase,
        tensordict_out: TensorDictBase | None = None,
        **kwargs,
    ) -> TensorDictBase:
        for module in self.module:
            tensordict = self._run_module(module, tensordict, **kwargs)
        if tensordict_out is not None:
            tensordict_out.update(tensordict, inplace=True)
            return tensordict_out
        return tensordict

In [9]:
combined_body = scenario_instance.combined_body
combined_policy_head = scenario_instance.combined_policy_head

In [10]:
combined_actor = TensorDictSequentialHooks(combined_body, combined_policy_head)

In [11]:
dataset = scenario_instance.train_dataset

In [12]:
dataset

GraphIsomorphismDataset(
    fields={
        adjacency: MemoryMappedTensor(shape=torch.Size([10000, 2, 11, 11]), device=cpu, dtype=torch.int32, is_shared=True),
        node_mask: MemoryMappedTensor(shape=torch.Size([10000, 2, 11]), device=cpu, dtype=torch.bool, is_shared=True),
        wl_score: MemoryMappedTensor(shape=torch.Size([10000]), device=cpu, dtype=torch.int32, is_shared=True),
        x: MemoryMappedTensor(shape=torch.Size([10000, 2, 11, 8]), device=cpu, dtype=torch.float32, is_shared=True),
        y: MemoryMappedTensor(shape=torch.Size([10000]), device=cpu, dtype=torch.int64, is_shared=True)},
    batch_size=torch.Size([10000]),
    device=None,
    is_shared=False)

In [13]:
max_num_nodes = dataset["x"].shape[-2]
max_num_nodes

11

## Looking at intermediate computations

In [14]:
dataloader = DataLoader(
    dataset=dataset,
    batch_size=BATCH_SIZE,
    generator=torch_generator,
)

In [15]:
statistic_collectors = {}

In [16]:
statistic_collectors["gnn_output"] = lambda storage: (
    storage["gnn_output"][:, 0].sum(dim=1) - storage["gnn_output"][:, 1].sum(dim=1)
).mean(dim=1)

In [17]:
statistic_collectors["gnn_output_rounded"] = lambda storage: (
    storage["gnn_output_rounded"][:, 0].sum(dim=1)
    - storage["gnn_output_rounded"][:, 1].sum(dim=1)
).mean(dim=1)

In [18]:
statistic_collectors["gnn_output_flatter"] = lambda storage: (
    storage["gnn_output_flatter"][:, :max_num_nodes].sum(dim=1)
    - storage["gnn_output_flatter"][:, max_num_nodes:].sum(dim=1)
).mean(dim=1)

In [19]:
statistic_collectors["transformer_input_initial"] = lambda storage: (
    storage["transformer_input_initial"][:, 2:max_num_nodes+2].sum(dim=1)
    - storage["transformer_input_initial"][:, max_num_nodes+2:].sum(dim=1)
).mean(dim=1)

In [20]:
statistic_collectors["transformer_input_pre_encoder"] = lambda storage: (
    storage["transformer_input_pre_encoder"][:, 2:max_num_nodes+2].sum(dim=1)
    - storage["transformer_input_pre_encoder"][:, max_num_nodes+2:].sum(dim=1)
).mean(dim=1)

In [21]:
statistic_collectors["transformer_input"] = lambda storage: (
    storage["transformer_input"][:, 2:max_num_nodes+2].sum(dim=1)
    - storage["transformer_input"][:, max_num_nodes+2:].sum(dim=1)
).mean(dim=1)

In [22]:
statistic_collectors["transformer_output_flatter"] = lambda storage: (
    storage["transformer_output_flatter"][:, 2:max_num_nodes+2].sum(dim=1)
    - storage["transformer_output_flatter"][:, max_num_nodes+2:].sum(dim=1)
).mean(dim=1)

In [23]:
statistic_collectors["node_level_repr_pre_encoder"] = lambda storage: (
    storage["node_level_repr_pre_encoder"][:, 0].sum(dim=1)
    - storage["node_level_repr_pre_encoder"][:, 1].sum(dim=1)
).mean(dim=1)

In [24]:
statistic_collectors["node_level_repr"] = lambda storage: (
    storage["node_level_repr"][:, 0].sum(dim=1)
    - storage["node_level_repr"][:, 1].sum(dim=1)
).mean(dim=1)

In [25]:
stats = {
    key: np.empty(len(dataset), dtype=np.float32)
    for key in statistic_collectors.keys()
}

In [26]:
storage = {}
hooks = GraphIsomorphismAgentHooks.create_recorder_hooks(storage, per_agent=True)

In [27]:
decision_logits = np.empty((len(dataset), 3), dtype=np.float32)
correct_decision = np.empty(len(dataset), dtype=bool)

In [28]:
for i, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
    batch["message"] = torch.zeros_like(batch["y"])
    batch["ignore_message"] = torch.ones_like(batch["y"], dtype=torch.bool)
    batch["round"] = torch.zeros(batch.batch_size, dtype=torch.int64)
    batch["decision_restriction"] = torch.zeros(batch.batch_size, dtype=torch.int64)
    actor_out = combined_actor(batch, hooks=hooks)
    batch_decision_logits = (
        actor_out["agents"]["decision_logits"][:, 0].detach().cpu().numpy()
    )
    decision_logits[i * BATCH_SIZE : (i + 1) * BATCH_SIZE] = batch_decision_logits
    batch_y = batch["y"].detach().cpu().numpy()
    correct_decision[i * BATCH_SIZE : (i + 1) * BATCH_SIZE] = (
        batch_y == batch_decision_logits.argmax(axis=1)
    )
    for key, collector in statistic_collectors.items():
        stats[key][i * BATCH_SIZE : (i + 1) * BATCH_SIZE] = (
            collector(storage["verifier"]).detach().cpu().numpy()
        )

100%|██████████| 79/79 [00:05<00:00, 14.96it/s]


In [29]:
actor_out

TensorDict(
    fields={
        adjacency: Tensor(shape=torch.Size([16, 2, 11, 11]), device=cpu, dtype=torch.int32, is_shared=False),
        agents: TensorDict(
            fields={
                decision_logits: Tensor(shape=torch.Size([16, 2, 3]), device=cpu, dtype=torch.float32, is_shared=False),
                graph_level_repr: Tensor(shape=torch.Size([16, 2, 2, 16]), device=cpu, dtype=torch.float32, is_shared=False),
                node_level_repr: Tensor(shape=torch.Size([16, 2, 2, 11, 16]), device=cpu, dtype=torch.float32, is_shared=False),
                node_selected_logits: Tensor(shape=torch.Size([16, 2, 22]), device=cpu, dtype=torch.float32, is_shared=False)},
            batch_size=torch.Size([16]),
            device=None,
            is_shared=False),
        decision_restriction: Tensor(shape=torch.Size([16]), device=cpu, dtype=torch.int64, is_shared=False),
        ignore_message: Tensor(shape=torch.Size([16]), device=cpu, dtype=torch.bool, is_shared=False),
   

In [30]:
max_wl_score = torch.max(dataset["wl_score"]).item()

In [31]:
means = {}
stds = {}
raw_data = {}
for wl_score in range(-1, max_wl_score + 1):
    if wl_score not in dataset["wl_score"]:
        means[wl_score] = {key: np.nan for key in stats.keys()}
        stds[wl_score] = {key: np.nan for key in stats.keys()}
        raw_data[wl_score] = {key: np.empty(0, dtype=np.float32) for key in stats.keys()}
        continue
    mask = (dataset["wl_score"] == wl_score).numpy()
    means[wl_score] = {key: stats[key][mask].mean() for key in stats.keys()}
    stds[wl_score] = {key: stats[key][mask].std() for key in stats.keys()}
    raw_data[wl_score] = {key: stats[key][mask] for key in stats.keys()}

In [32]:
console = Console()

table = Table()
table.add_column("Score")
for key in stats.keys():
    table.add_column(key)

for key, mean, std in zip(means.keys(), means.values(), stds.values()):
    table.add_row(str(key), *[f"{mean[key]:.4f}" for key in stats.keys()])

console.print(table)


┏━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Score ┃ gnn_outp… ┃ gnn_outp… ┃ gnn_outp… ┃ transfor… ┃ transfor… ┃ transfo… ┃ transfor… ┃ node_le… ┃ node_lev… ┃
┡━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━┩
│ -1    │ -0.0000   │ -0.0000   │ -0.0000   │ -0.0000   │ -0.0000   │ -0.0000  │ -0.0000   │ -0.0000  │ 0.0000    │
│ 0     │ nan       │ nan       │ nan       │ nan       │ nan       │ nan      │ nan       │ nan      │ nan       │
│ 1     │ 0.0062    │ 0.0062    │ 0.0062    │ 0.0062    │ 0.0052    │ -0.0030  │ 0.0000    │ 0.0000   │ -0.0002   │
│ 2     │ 0.0001    │ 0.0001    │ 0.0001    │ 0.0001    │ 0.0001    │ -0.0001  │ -0.0000   │ -0.0000  │ -0.0001   │
│ 3     │ -0.0000   │ -0.0000   │ -0.0000   │ -0.0000   │ -0.0000   │ -0.0000  │ -0.0000   │ -0.0000  │ 0.0000    │
│ 4     │ -0.0000   │ -0.0000   │ -0.0000   │ -0.0000   │ -0.0000   │ -0.0000  │ -0.0000   │ -0.0000  │ -0.0000   │
│ 5     │ -0.0000   │ -0.0000   │ -0.0000   │ -0.0000   │ -0.0000   │ -0.0000  │ 0.0000    │ 0.0000   │ -0.0000   │
└───────┴───────────┴───────────┴───────────┴───────────┴───────────┴──────────┴───────────┴──────────┴───────────┘

In [33]:
gnn_output_mean = np.array([means[wl_score]["gnn_output"] for wl_score in range(-1, max_wl_score + 1)])
gnn_output_std = np.array([stds[wl_score]["gnn_output"] for wl_score in range(-1, max_wl_score + 1)])
x = list(range(-1, max_wl_score + 1))

fig = go.Figure(data=go.Scatter(x=x, y=gnn_output_mean, error_y=dict(type='data', array=gnn_output_std)))
fig.update_layout(title="gnn_output_mean", xaxis_title="wl_score", yaxis_title="Mean")
fig.show()

In [34]:
gnn_output_rounded_mean = np.array([means[wl_score]["gnn_output_rounded"] for wl_score in range(-1, max_wl_score + 1)])
gnn_output_rounded_std = np.array([stds[wl_score]["gnn_output_rounded"] for wl_score in range(-1, max_wl_score + 1)])
x = list(range(-1, max_wl_score + 1))

fig = go.Figure(data=go.Scatter(x=x, y=gnn_output_rounded_mean, error_y=dict(type='data', array=gnn_output_rounded_std)))
fig.update_layout(title="gnn_output_rounded_mean", xaxis_title="wl_score", yaxis_title="Mean")
fig.show()

In [35]:
node_level_repr_pre_encoder_mean = np.array([means[wl_score]["node_level_repr_pre_encoder"] for wl_score in range(-1, max_wl_score + 1)])
node_level_repr_pre_encoder_std = np.array([stds[wl_score]["node_level_repr_pre_encoder"] for wl_score in range(-1, max_wl_score + 1)])
x = list(range(-1, max_wl_score + 1))

fig = go.Figure(data=go.Scatter(x=x, y=node_level_repr_pre_encoder_mean, error_y=dict(type='data', array=node_level_repr_pre_encoder_std)))
fig.update_layout(title="node_level_repr_pre_encoder_mean", xaxis_title="wl_score", yaxis_title="Mean")
fig.show()

In [36]:
raw_data.keys()

dict_keys([-1, 0, 1, 2, 3, 4, 5])

In [37]:
px.histogram(raw_data[3]["node_level_repr_pre_encoder"])

## Combining the body and policy head

In [38]:
decision_logits.argmax(axis=1).mean()

2.0

In [39]:
correct_decision.mean()

0.0